In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import sys

sys.path.append(os.getcwd())

from training.policy_network.model import ChessPolicyNet as Model, INPUT_CHANNELS

MODEL_PATH = "models/policy_network/BetaChess.pt"
ONNX_PATH = "models/policy_network/BetaChess.onnx"


def export():
    try:
        import onnx
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
        import onnx

    print(f"Inicjalizacja modelu ResNet (BetaChess)...")
    
    model = Model() 
    
    print(f"Wczytywanie wag z {MODEL_PATH}...")
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
    except Exception as e:
        print(f"Błąd ładowania wag: {e}")
        return

    model.eval()

    print(f"Tworzenie dummy_input o wymiarach [1, {INPUT_CHANNELS}, 8, 8]...")
    dummy_input = torch.randn(1, INPUT_CHANNELS, 8, 8, requires_grad=True)

    print(f"Eksportowanie do {ONNX_PATH}...")
    torch.onnx.export(model, 
        dummy_input, 
        ONNX_PATH, 
        export_params=True,        
        opset_version=11,          
        do_constant_folding=True,
        input_names=['input'], 
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    
    print(f"SUKCES! Wyeksportowano model do {ONNX_PATH}")

if __name__ == "__main__":
    export()

Inicjalizacja modelu ResNet (BetaChess)...
Wczytywanie wag z models/policy_network/BetaChess.pt...
Tworzenie dummy_input o wymiarach [1, 69, 8, 8]...
Eksportowanie do models/policy_network/BetaChess.onnx...


/tmp/ipykernel_244251/3574931589.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))


SUKCES! Wyeksportowano model do models/policy_network/BetaChess.onnx
